# Section 2 Risk Factor Extraction

In section 1, we have downloaded the fincancial reports from 2019Q1 to 2023Q3. In this section, we are going to extract the risk factor text from each of these files. According to the requirement of the 10-K form given by SEC (https://www.sec.gov/files/form10-k.pdf), there is an section named "Item 1A. Risk Factors" included in the form  which shows the potential risks faced by the company. Therefore, it is important for us to filter and cleanse the text from this part so that we can use it for further analysis.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
import html
import warnings
warnings.filterwarnings("ignore")

# 2.2 Bank Industry (6022)

In this section, we only focus on companies in Bank Industry.

## 2.2.1 Industry 6022

### 1. Read in the files and store in a dictionary

In [2]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Bank Industry/Industry_6022/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

In [3]:
filePair_dict.keys()

dict_keys(['0000007789-23-000013.txt', '0000022356-23-000017.txt', '0000034782-23-000043.txt', '0000035527-23-000122.txt', '0000039311-23-000035.txt', '0000040729-23-000005.txt', '0000073124-23-000049.txt', '0000093751-23-000489.txt', '0000102212-23-000016.txt', '0000314489-23-000010.txt', '0000318300-23-000133.txt', '0000351569-23-000005.txt', '0000356171-23-000010.txt', '0000701347-23-000008.txt', '0000705432-23-000023.txt', '0000706129-23-000034.txt', '0000706863-23-000016.txt', '0000711669-23-000041.txt', '0000714395-23-000008.txt', '0000714562-23-000004.txt', '0000715072-23-000054.txt', '0000715579-23-000015.txt', '0000716605-23-000007.txt', '0000719220-23-000018.txt', '0000719739-23-000021.txt', '0000721994-23-000014.txt', '0000723646-23-000018.txt', '0000726601-23-000009.txt', '0000730708-23-000017.txt', '0000732417-23-000013.txt', '0000736772-23-000023.txt', '0000737468-23-000008.txt', '0000737875-23-000015.txt', '0000759944-23-000029.txt', '0000763563-23-000037.txt', '00007769

### 2. Extract the raw risk factor text

In [4]:
def remove_tags(text):
    # replace the tag included by <> to a \ 
    pattern1 = r'<[^>]*>'
    cleanText = re.sub(pattern1, "|", text)
    
    # replace the multiple continued \ to a single \
    cleanText = re.sub(r'\s*\|+\s*', '|', cleanText)
    cleanText = re.sub(r'\|+', '|', cleanText)
    
    # replace the unicode
    cleanText = html.unescape(cleanText)
    
    # remove the line break
    cleanText = cleanText.replace('\n', '')
    # remove the pure numbers text between \
    cleanText = re.sub(r'\|(\d+)\|', '|', cleanText)
    # remove the unrecognized string
    cleanText = re.sub(r'|\xa0\|', '', cleanText)
    cleanText = re.sub(r'|\xa0|', '', cleanText)
    return cleanText

In [5]:
def separate_main_body(text):
    lastContentInTC_lst = [">form 10-k summary<", ">item 16. form 10-k summary<", r">exhibits, financial statement schedules<", 
                           r">exhibits and financial statement schedules<", r"exhibits and financial statement schedules <",
                           ">exhibits<"]
    start_idx = -1
    for content in lastContentInTC_lst:
        if content in text.lower():
            start_idx = text.lower().find(content)
            break
            
    if start_idx == -1:
        return False
    
    mainBody = text[start_idx:]
    mainBody = html.unescape(mainBody)
    return mainBody

In [6]:
def extract_risk_factors(mainBody, fileName):
    # match the risk factor
    keywordMatch = re.findall(r">item 1a.\s+risk factors\.?\<", mainBody.lower())
    
    # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
    if not keywordMatch:
        keyword = seperated_keyword_rematch(mainBody)
        
    # if match the pattern successfully, just keep the keyword of risk factor
    else:
        keyword = keywordMatch[0]
    
    # check if match successfully
    if not keyword:
        return False
    keywordCheck_lst = ["item", r"item 1a\.?", "risk", "factors", "risk factors"]
    if keyword in keywordCheck_lst:
        print("Warning: It is better to check whether the extraction is correct manually for file", {fileName})
    
    # match the next section of the risk factor
    nextSectionPattern_lst = [r">item 1b.\s+unresolved staff comments\.?\<", r">item 2.\s+description of property\.?\<"]
    for pattern in nextSectionPattern_lst:
        nextSectionMatch = re.findall(pattern, mainBody.lower())
        
        # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
        if not nextSectionMatch:
            nextSection = seperated_next_section_rematch(mainBody, pattern)
            # if re-match successfully, jump out of the loop
            if nextSection:
                break
        
        # if match the pattern successfully, just keep the next section keyword and jump out of the loop
        else:
            nextSection = nextSectionMatch[0]
            break
    
    # check if match successfully
    if not nextSection:
        return False
    
    # if match successfully, extract the risk factor
    start_idx = mainBody.lower().find(keyword) # start index is the position of the keyword occured in the main body
    end_idx = mainBody.lower().find(nextSection) # end index is the position of the next section begin in the main body
    riskFactors = mainBody[start_idx: end_idx]
    return riskFactors

In [7]:
def seperated_keyword_rematch(mainBody):
    cleanedMainBody = remove_tags(mainBody)
    keywordMatch = re.findall(r"i\|?t\|?e\|?m\|?\s+\|?1\|?a\|?.\|?\s+\|?r\|?i\|?s\|?k\|?\s+\|?f\|?a\|?c\|?t\|?o\|?r\|?s\.?", cleanedMainBody.lower())
    
    # check if re-match the keyword of risk factor successfully
    if not keywordMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the risk factor
    reMatchedKeyword = max(keywordMatch[0].split('|'), key=len)
    return reMatchedKeyword
    
def seperated_next_section_rematch(mainBody, nextSectionPattern):
    # cleanse the main body text
    cleanedMainBody = remove_tags(mainBody)
    
    # edit the pattern 
    edittedPattern = nextSectionPattern.replace(r">", "").replace(r"\s+", " ").replace(r"\.?\<", "")
    edittedPattern = "\|?".join(edittedPattern)
    edittedPattern = r"{}\.?".format(edittedPattern)
    pattern = edittedPattern.replace(" ", "\s?+")
    nextSectionMatch = re.findall(pattern, cleanedMainBody.lower())
    
    # check if re-match the keyword of next section successfully
    if not nextSectionMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the next section 
    reMatchedNextSection = max(nextSectionMatch[0].split('|'), key=len)
    return reMatchedNextSection

In [8]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6022/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000039311-23-000035.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000040729-23-000005.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000073124-23-000049.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000102212-23-000016.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000318300-23-000133.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000351569-23-000005.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000356171-23-000010.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000705432-23-000023.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000711669-23-000041.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000071

The extraction is failed for {'0001558370-23-001892.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-002028.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-002263.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-002423.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-002607.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-002793.txt'} because of the error in seperating main body.
The extraction is failed for {'0001558370-23-002993.txt'} because of the error in seperating main body.
The extraction is failed for {'0001558370-23-003139.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-003238.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-2

### 3. Extract the subjects from raw risk factors text

In [9]:
from bs4 import BeautifulSoup

def extract_subjects(riskFactors, subjectFormat):
    soup = BeautifulSoup(riskFactors, 'html.parser')
    subject_lst = []
    for span in soup.find_all('span'):
        style = span.get('style')
        if style and subjectFormat in style:
            subject_lst.append(span.get_text())
    return subject_lst

In [10]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6022/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [11]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6022/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0000723646-23-000018.txt'}
The subject fail to extract for {'0001140361-23-009386.txt'}
The subject fail to extract for {'0001140361-23-013727.txt'}
The subject fail to extract for {'0001140361-23-014828.txt'}
The subject fail to extract for {'0001171843-23-001247.txt'}
The subject fail to extract for {'0001193125-23-056454.txt'}
The subject fail to extract for {'0001193125-23-074092.txt'}
The subject fail to extract for {'0001213900-23-018266.txt'}
The subject fail to extract for {'0001315399-23-000020.txt'}
The subject fail to extract for {'0001390777-23-000033.txt'}
The subject fail to extract for {'0001437749-23-004952.txt'}
The subject fail to extract for {'0001437749-23-005213.txt'}
The subject fail to extract for {'0001437749-23-005878.txt'}
The subject fail to extract for {'0001437749-23-006190.txt'}
The subject fail to extract for {'0001437749-23-007584.txt'}
The subject fail to extract for {'0001437749-23-007608.txt'}
The subject fail to extr

---

## 2.2.2 Industry 6021

### 1. Read in the files and store in a dictionary

In [12]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Bank Industry/Industry_6021/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

### 2. Extract the raw risk factor text

In [13]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6021/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000018349-23-000061.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000019617-23-000231.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000037808-23-000005.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000049196-23-000020.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000072971-23-000071.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000083246-23-000004.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000092230-23-000034.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000712537-23-000050.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000713676-23-000020.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000072

### 3. Extract the subjects from raw risk factors text

In [14]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6021/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [15]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Bank Industry/Industry_6021/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001140361-23-011796.txt'}
The subject fail to extract for {'0001171843-23-001285.txt'}
The subject fail to extract for {'0001174947-23-000382.txt'}
The subject fail to extract for {'0001206774-23-000172.txt'}
The subject fail to extract for {'0001437749-23-006097.txt'}
The subject fail to extract for {'0001437749-23-006159.txt'}
The subject fail to extract for {'0001493152-23-009718.txt'}
The subject fail to extract for {'0001558370-23-004728.txt'}
The subject fail to extract for {'0001562762-23-000110.txt'}
The subject fail to extract for {'0001654954-23-003557.txt'}
